In [1]:
! source venv/bin/activate

In [2]:
# ! pip install litellm --break-system-packages

In [ ]:
from litellm import completion
import json
import strands, strands_tools

MODEL = "ollama/devstral-small-2:24b-cloud"   # change to the model you have pulled in Ollama


In [4]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful agents that explains things in simple language"
    },
    {
        "role": "user",
        "content": "What's recursion?"
    }
]

In [5]:
response = completion(model=MODEL, messages=messages)

In [6]:
print (response.choices[0].message.content)

Recursion is a way of solving a problem by breaking it down into smaller, similar problems. It's like a loop that calls itself to solve a smaller version of the same problem until it reaches a simple case that can be solved directly.

### Simple Example:
Imagine you have a stack of plates, and you want to take them all off. You can do it by:
1. Taking the top plate off.
2. Then, you have a smaller stack (without the top plate).
3. Repeat the same process (taking the top plate off) until no plates are left.

This is recursion! The "smaller stack" is the smaller version of the original problem.

### Key Parts of Recursion:
1. **Base Case**: The simplest case that stops the recursion (e.g., no plates left).
2. **Recursive Case**: The problem calls itself with a smaller input (e.g., taking one plate off).

### Why Use Recursion?
- It makes some problems easier to understand (like tree structures or divide-and-conquer problems).
- It can be elegant and concise.

### Example in Code (Factori

In [7]:
# Define tools the LLM can use (JSON Schema format)
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City name, e.g., 'San Francisco, CA'",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "Temperature unit",
                    },
                },
                "required": ["location"],
            },
        },
    }
]

In [8]:
messages = [{"role": "user", "content": "What's the weather like in Madison, WI"}]

response = completion(
    model=MODEL,
    messages = messages,
    tools = tools,
)

In [9]:
print(response.choices[0].message)

Message(content='```json\n{"name": "get_weather", "arguments": {"location": "Madison, WI"}}\n```', role='assistant', tool_calls=None, function_call=None, provider_specific_fields=None)


In [10]:
tools = [
        {
            "type": "function",
            "function": {
                "name": "calculate",
                "description": "Perform a mathematical calculation",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "expression": {
                            "type": "string",
                            "description": "Math expression to evaluate, e.g., '2 + 2 * 3'",
                        }
                    },
                    "required": ["expression"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_exchange_rate",
                "description": "Get the exchange rate between two currencies",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "from_currency": {
                            "type": "string",
                            "description": "Source currency code, e.g., 'USD'",
                        },
                        "to_currency": {
                            "type": "string",
                            "description": "Target currency code, e.g., 'EUR'",
                        },
                    },
                    "required": ["from_currency", "to_currency"],
                },
            },
        },
    ]

In [11]:
# Simulated tool implementations
def execute_tool(name: str, arguments: dict) -> str:
    """Execute a tool and return the result as a string."""
    if name == "calculate":
        try:
            # WARNING: eval is dangerous!
            result = eval(arguments["expression"])
            return json.dumps({"result": result})
        except Exception as e:
            return json.dumps({"error": str(e)})

    elif name == "get_exchange_rate":
        # Simulated exchange rates
        rates = {
            ("USD", "EUR"): 0.92,
            ("USD", "JPY"): 149.50,
            ("EUR", "USD"): 1.09,
        }
        key = (arguments["from_currency"], arguments["to_currency"])
        rate = rates.get(key, 1.0)
        return json.dumps(
            {
                "from": arguments["from_currency"],
                "to": arguments["to_currency"],
                "rate": rate,
            }
        )

    return json.dumps({"error": "Unknown tool"})


In [12]:
 # Start the conversation
messages = [
    {
        "role": "user",
        "content": "I have 150 USD. How much is that in EUR? Then calculate what 15% tip would be on that EUR amount.",
    }
]

In [13]:
print(f"\nUser: {messages[0]['content']}")
print("\n--- Agentic Loop Starting ---")

# The agentic loop
for iteration in range(1, 11):
    print(f"\n[Iteration {iteration}]")

    response = completion(model=MODEL, messages=messages, tools=tools)
    assistant_message = response.choices[0].message
    messages.append(assistant_message.model_dump())

    if not assistant_message.tool_calls:
        print("\n--- Agentic Loop Complete ---")
        print(f"\nFinal Answer: {assistant_message.content}")
        break

    for tc in assistant_message.tool_calls:
        args = json.loads(tc.function.arguments)
        print(f"  Tool call: {tc.function.name}({args})")
        result = execute_tool(tc.function.name, args)
        print(f"  Result: {result}")
        messages.append({"role": "tool", "tool_call_id": tc.id, "content": result})
else:
    print("\nWarning: Max iterations reached!")


User: I have 150 USD. How much is that in EUR? Then calculate what 15% tip would be on that EUR amount.

--- Agentic Loop Starting ---

[Iteration 1]

--- Agentic Loop Complete ---

Final Answer: ```json
{"name": "get_exchange_rate", "arguments": {"from_currency": "USD", "to_currency": "EUR"}}
```


In [14]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "bash",
            "description": "Run a bash command and return stdout/stderr",
            "parameters": {
                "type": "object",
                "properties": {
                    "command": {"type": "string", "description": "The bash command to run"}
                },
                "required": ["command"],
            },
        },
    }
]


import subprocess
def run_bash(command: str) -> str:
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    return result.stdout + result.stderr or "(no output)"

messages = [
    {"role": "system", "content": "You are a helpful assistant that can run bash commands. Be concise."},
    {"role": "user", "content": "List the Python files in the current directory and count them."},
]

print(f"\nUser: {messages[1]['content']}")

for _ in range(10):
    response = completion(model=MODEL, messages=messages, tools=tools)
    msg = response.choices[0].message
    messages.append(msg.model_dump())

    if not msg.tool_calls:
        print(f"\nAssistant: {msg.content}")
        break

    for tc in msg.tool_calls:
        cmd = json.loads(tc.function.arguments)["command"]
        print(f"\n$ {cmd}")
        output = run_bash(cmd)
        print(output)
        messages.append({"role": "tool", "tool_call_id": tc.id, "content": output})


User: List the Python files in the current directory and count them.

Assistant: {"bash": {"command": "ls -1 *.py | wc -l"}}
{"bash": {"command": "ls -1 *.py"}}


In [30]:
import subprocess
from strands import Agent, tool
from strands.models import ollama

@tool
def bash(cmd: str) -> str:
    p = subprocess.run(["bash", "-lc", cmd], capture_output=True, text=True)
    return f"exit_code={p.returncode}\nSTDOUT:\n{p.stdout}\nSTDERR:\n{p.stderr}"

model = ollama.OllamaModel(
    model_id="devstral-small-2:24b-cloud",              # must match `ollama list`
    host="http://localhost:11434",       # default Ollama host
)

agent = Agent(model=model, tools=[bash])
print(agent("Use bash to list files in the directory: venv."))



Tool #1: bash
Here is the list of files and directories in the `venv` directory:

- `bin`: A directory (likely contains executable scripts).
- `include`: A directory (likely contains header files).
- `lib`: A directory (likely contains library files).
- `lib64`: A symbolic link pointing to `lib`.
- `pyvenv.cfg`: A configuration file for the virtual environment.
- `share`: A directory (likely contains shared data).Here is the list of files and directories in the `venv` directory:

- `bin`: A directory (likely contains executable scripts).
- `include`: A directory (likely contains header files).
- `lib`: A directory (likely contains library files).
- `lib64`: A symbolic link pointing to `lib`.
- `pyvenv.cfg`: A configuration file for the virtual environment.
- `share`: A directory (likely contains shared data).

